- pandas: Used for data manipulation and analysis.

- numpy: Used for numerical operations.

In [1]:
import pandas as pd
import numpy as np

- Load the dataset from an Excel file named 'MOOD TRAK.xlsx' into a DataFrame.

- Display the first few rows of the DataFrame to inspect the data.

In [2]:
df=pd.read_excel('MOOD TRAK.xlsx')
df.head()

,Time Stamp,Activities,Duration,Quality,Note
0,2024-05-01,Sleep,300,3,Bangun pagi untuk sholat subuh.
1,2024-05-01,Self Care,30,4,"Mandi dan bersiap, air dingin di pagi hari tid..."
2,2024-05-01,Eating,30,4,Sarapan pagi dengan roti bakar dan susu.
3,2024-05-01,Workout,60,5,Jogging sekitar kos-kosan. Bertemu banyak anak...
4,2024-05-01,Traveling,30,5,"Berangkat ke kampus, perjalanan sekitar 2 km d..."


- Drop the 'Note' column as it is not relevant to the model.

- Convert the 'Time Stamp' column to datetime format and keep only the date part.

- Convert 'Start' and 'End' columns to datetime format and keep only the time part.

- Calculate the duration of each activity as the difference between 'End' and 'Start' times.

- Adjust for activities that span past midnight by adding a day to negative durations.

- Convert the duration from timedelta to minutes.



In [3]:
df=pd.read_excel('MOOD TRAK.xlsx')
df.head()

,Time Stamp,Activities,Duration,Quality,Note
0,2024-05-01,Sleep,300,3,Bangun pagi untuk sholat subuh.
1,2024-05-01,Self Care,30,4,"Mandi dan bersiap, air dingin di pagi hari tid..."
2,2024-05-01,Eating,30,4,Sarapan pagi dengan roti bakar dan susu.
3,2024-05-01,Workout,60,5,Jogging sekitar kos-kosan. Bertemu banyak anak...
4,2024-05-01,Traveling,30,5,"Berangkat ke kampus, perjalanan sekitar 2 km d..."


In [6]:
# Drop 'Note' columns because its not used in this model
df_new=df.drop(columns=['Note'])
df_new['Time Stamp'] = pd.to_datetime(df_new['Time Stamp']).dt.date

Group the data by 'Time Stamp' and 'Activities' and calculate the sum of durations for each combination.

Pivot the aggregated data to have 'Time Stamp' as the index and activities as columns.

Replace missing values with 0 to handle days when certain activities were not performed.



In [7]:
# Aggregate daily data
daily_summary = df_new.groupby(['Time Stamp', 'Activities']).agg({'Duration': 'sum'}).reset_index()

daily_summary_pivot = daily_summary.pivot(index='Time Stamp', columns='Activities', values='Duration').fillna(0)
daily_summary_pivot.reset_index(inplace=True)
daily_summary_pivot

Activities,Time Stamp,Dating,Eating,Entertainment,Self Care,Sleep,Study,Traveling,Work,Workout
0,2024-05-01,0.0,105.0,165.0,45.0,300.0,630.0,135.0,0.0,60.0
1,2024-05-02,260.0,135.0,0.0,45.0,275.0,270.0,145.0,0.0,75.0
2,2024-05-03,0.0,120.0,180.0,60.0,480.0,480.0,60.0,0.0,60.0
3,2024-05-04,210.0,90.0,300.0,45.0,795.0,0.0,0.0,0.0,0.0
4,2024-05-05,150.0,85.0,465.0,0.0,300.0,240.0,200.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
93,2024-08-02,120.0,120.0,75.0,30.0,180.0,90.0,75.0,465.0,45.0
94,2024-08-03,120.0,120.0,75.0,30.0,180.0,90.0,75.0,435.0,45.0
95,2024-08-04,120.0,120.0,75.0,30.0,180.0,90.0,75.0,435.0,45.0
96,2024-08-05,120.0,120.0,75.0,30.0,180.0,90.0,75.0,465.0,45.0


In [13]:
# Define normal ranges for activities
normal_ranges = {
    'Dating': (30, 60),
    'Eating': (90, 120),
    'Entertainment': (60, 180),
    'Self Care': (30, 60),
    'Sleep': (420, 540),
    'Study': (120, 240),
    'Traveling': (60, 120),
    'Work': (420, 540),
    'Workout': (30, 60)
}

def generate_recommendations(df, normal_ranges):
    recommendations = []

    for index, row in df.iterrows():
        daily_recommendations = {'Time Stamp': row['Time Stamp']}
        for activity, (min_time, max_time) in normal_ranges.items():
            actual_time = row[activity]
            if actual_time < min_time:
                daily_recommendations[activity] = f"Increase on your {activity} by {min_time - actual_time} minutes"
            elif actual_time > max_time:
                daily_recommendations[activity] = f"Decrease on your {activity} by {actual_time - max_time} minutes"
            else:
                daily_recommendations[activity] = "Optimal"
        recommendations.append(daily_recommendations)

    return pd.DataFrame(recommendations)

# Generate recommendations
recommendations_df = generate_recommendations(daily_summary_pivot, normal_ranges)
recommendations_df

,Time Stamp,Dating,Eating,Entertainment,Self Care,Sleep,Study,Traveling,Work,Workout
0,2024-05-01,Increase on your Dating by 30.0 minutes,Optimal,Optimal,Optimal,Increase on your Sleep by 120.0 minutes,Decrease on your Study by 390.0 minutes,Decrease on your Traveling by 15.0 minutes,Increase on your Work by 420.0 minutes,Optimal
1,2024-05-02,Decrease on your Dating by 200.0 minutes,Decrease on your Eating by 15.0 minutes,Increase on your Entertainment by 60.0 minutes,Optimal,Increase on your Sleep by 145.0 minutes,Decrease on your Study by 30.0 minutes,Decrease on your Traveling by 25.0 minutes,Increase on your Work by 420.0 minutes,Decrease on your Workout by 15.0 minutes
2,2024-05-03,Increase on your Dating by 30.0 minutes,Optimal,Optimal,Optimal,Optimal,Decrease on your Study by 240.0 minutes,Optimal,Increase on your Work by 420.0 minutes,Optimal
3,2024-05-04,Decrease on your Dating by 150.0 minutes,Optimal,Decrease on your Entertainment by 120.0 minutes,Optimal,Decrease on your Sleep by 255.0 minutes,Increase on your Study by 120.0 minutes,Increase on your Traveling by 60.0 minutes,Increase on your Work by 420.0 minutes,Increase on your Workout by 30.0 minutes
4,2024-05-05,Decrease on your Dating by 90.0 minutes,Increase on your Eating by 5.0 minutes,Decrease on your Entertainment by 285.0 minutes,Increase on your Self Care by 30.0 minutes,Increase on your Sleep by 120.0 minutes,Optimal,Decrease on your Traveling by 80.0 minutes,Increase on your Work by 420.0 minutes,Increase on your Workout by 30.0 minutes
...,...,...,...,...,...,...,...,...,...,...
93,2024-08-02,Decrease on your Dating by 60.0 minutes,Optimal,Optimal,Optimal,Increase on your Sleep by 240.0 minutes,Increase on your Study by 30.0 minutes,Optimal,Optimal,Optimal
94,2024-08-03,Decrease on your Dating by 60.0 minutes,Optimal,Optimal,Optimal,Increase on your Sleep by 240.0 minutes,Increase on your Study by 30.0 minutes,Optimal,Optimal,Optimal
95,2024-08-04,Decrease on your Dating by 60.0 minutes,Optimal,Optimal,Optimal,Increase on your Sleep by 240.0 minutes,Increase on your Study by 30.0 minutes,Optimal,Optimal,Optimal
96,2024-08-05,Decrease on your Dating by 60.0 minutes,Optimal,Optimal,Optimal,Increase on your Sleep by 240.0 minutes,Increase on your Study by 30.0 minutes,Optimal,Optimal,Optimal
